In [16]:
# Imports
import numpy as np
import pandas as pd
import tensorflow as tf
import os

from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras import layers, models

In [17]:
# Data comes from: https://www.kaggle.com/competitions/dog-breed-identification/data
# Later I put them into my Google Dysc

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Paths
labels_csv = pd.read_csv("drive/MyDrive/Data/labels.csv")

train_dir = "/content/drive/MyDrive/Data/train/"
test_dir  = "/content/drive/MyDrive/Data/test/"

In [19]:
labels_csv.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [20]:
labels_csv.describe()


,id,breed
count,10222,10222
unique,10222,120
top,fff43b07992508bc822f33d8ffd902ae,scottish_deerhound
freq,1,126


In [21]:
labels_csv["path"] = labels_csv["id"].apply(
    lambda x: os.path.join(train_dir, f"{x}.jpg")
)

In [22]:
# Encode
labels = labels_csv["breed"].to_numpy()
unique_breeds = np.unique(labels)
breed_to_num = {b: i for i, b in enumerate(unique_breeds)}
labels_num = labels_csv["breed"].map(breed_to_num).to_numpy()

In [23]:
# Prepare training arrays
train_image_paths = labels_csv["path"].to_numpy()
train_labels = labels_csv["breed"].map(breed_to_num).to_numpy()

In [24]:
# Image loader
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (384, 384))
    img = img / 255.0
    return img, label

In [26]:
# Dataset
train_ds = tf.data.Dataset.from_tensor_slices((train_image_paths, train_labels))
train_ds = train_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)

In [27]:
# Model
base_model = EfficientNetV2B0(
    include_top=False,
    weights="imagenet",
    input_shape=(384, 384, 3)
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(len(unique_breeds), activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# Training
model.fit(train_ds, epochs=5)

base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(train_ds, epochs=10)

In [29]:
# Prepering for test
test_files = sorted([
    os.path.join(test_dir, f)
    for f in os.listdir(test_dir)
    if f.endswith(".jpg")
])

In [30]:
def load_test_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (384, 384))
    img = img / 255.0
    return img

In [31]:
test_ds = tf.data.Dataset.from_tensor_slices(test_files)
test_ds = test_ds.map(lambda x: load_test_image(x), num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Predict
preds = model.predict(test_ds)
pred_labels = preds.argmax(axis=1)

In [ ]:
# Decode
inverse_map = {v: k for k, v in breed_to_num.items()}
final_breeds = [inverse_map[i] for i in pred_labels]

In [33]:
# Building Submission File

submission = pd.DataFrame({
    "id": [os.path.splitext(os.path.basename(p))[0] for p in test_files],
    "breed": final_breeds
})

submission.to_csv("submission.csv", index=False)

submission.head()